# Google Earth Engine Data Extract

Author [@ricoen](https://github.com/ricoen)<br>
Please cite me if you use this code :)

In [1]:
# Import libraries
import ee
import geemap as geemap
import os
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as dt, date
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON

In [2]:
# Initialize Google Earth Engine and Geemap
geemap.Map()


Successfully saved authorization token.


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [3]:
# Get AOI GeoJSON
def get_geojson(file_path):
    json_path = os.path.abspath(file_path)

    with open(json_path) as f:
        aoi_json = json.load(f)

    features = aoi_json['features']
    fc = ee.FeatureCollection(features)

    return fc

In [4]:
# Get image/raster data
def get_image(image):
    data = ee.ImageCollection(image)

    return data

In [5]:
# Clip raster
def clip_raster(fc, data):
    image_clip = data.clipToCollection(fc)

    return image_clip

In [6]:
# Calculate Zonal Statistic and export it as CSV
def calculate_land_cover(fc, image_clip):
    parent_dir = os.path.join('data')
    land_cover_stats = os.path.join(parent_dir, 'land_cover.csv')

    geemap.zonal_statistics_by_group(
        image_clip,
        fc,
        land_cover_stats,
        statistics_type='PERCENTAGE',
        denominator=1000000,
        decimal_places=2,
    )

In [7]:
# Get image series data
def image_series(start, latest, image, bands):
    images = ee.ImageCollection(image)\
        .select(bands)\
            .filterDate(start, latest)

    return images

In [8]:
# Reduce image region
def reduce_region(image):
    image_dict = image.reduceRegion(ee.Reducer.mean(), geometry=feature_collection)
    
    return image.set(image_dict)

In [9]:
# Filter image data
def filter_image(image):
    image_fc = ee.FeatureCollection(image.map(reduce_region))
    image_filtered = image_fc.filter(ee.Filter.notNull(image.first().bandNames()))

    return image_filtered

In [10]:
# Convert feature collection to dictionary
def fc_to_dict(fc):
    prop_names = fc.first().propertyNames()
    prop_lists = fc.reduceColumns(
        reducer=ee.Reducer.toList().repeat(prop_names.size()),
        selectors=prop_names).get('list')

    return ee.Dictionary.fromLists(prop_names, prop_lists)

In [11]:
# Convert dictionary to dataframe
def dict_to_df(fc):
    fc_dict = fc_to_dict(fc).getInfo()
    fc_df = pd.DataFrame(fc_dict)

    return fc_df

In [17]:
# Call functions
# global feature_collection
feature_collection = get_geojson(
    file_path='data/kota_malang/malang.geojson'
    )
image_data = get_image(
    image='ESA/WorldCover/v200'
    ).first()
clipped_image = clip_raster(
    fc=feature_collection, 
    data=image_data
    )
    
calculate_land_cover(
    fc=feature_collection, 
    image_clip=clipped_image
    )

start_date = '2023-01-01'
latest_date = date.today().strftime('%Y-%m-%d')

images = image_series(
    start=start_date, latest=latest_date, 
    image='NASA/GLDAS/V021/NOAH/G025/T3H', 
    bands='Tair_f_inst'
    )
fc_filtered = filter_image(
    image=images
    )

Computing ... 
Generating URL ...
Please wait ...
Data downloaded to /home/ricoen/Projects/google-ee-data-extraction/data/land_cover.csv


In [18]:
dict_to_df(fc_filtered)

,Tair_f_inst,end_hour,start_hour,status,system:asset_size,system:band_names,system:bands,system:footprint,system:id,system:index,system:time_end,system:time_start,system:version
0,292.792227,3,0,permanent,25095776,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20230101_0000,A20230101_0000,1672542000000,1672531200000,1.681593e+15
1,295.164350,6,3,permanent,24927996,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20230101_0300,A20230101_0300,1672552800000,1672542000000,1.681593e+15
2,298.299421,9,6,permanent,25503123,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20230101_0600,A20230101_0600,1672563600000,1672552800000,1.681593e+15
3,295.487587,12,9,permanent,25856870,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20230101_0900,A20230101_0900,1672574400000,1672563600000,1.681593e+15
4,293.185316,15,12,permanent,25872287,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20230101_1200,A20230101_1200,1672585200000,1672574400000,1.681594e+15
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,297.164348,12,9,early,26066199,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20231211_0900,A20231211_0900,1702296000000,1702285200000,1.702694e+15
2756,293.445233,15,12,early,26072794,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20231211_1200,A20231211_1200,1702306800000,1702296000000,1.702695e+15
2757,292.851942,18,15,early,25808332,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20231211_1500,A20231211_1500,1702317600000,1702306800000,1.702695e+15
2758,291.786064,21,18,early,25608046,[Tair_f_inst],{'Tair_f_inst': {'data_type': {'type': 'PixelT...,"{'type': 'LinearRing', 'coordinates': [[-180, ...",NASA/GLDAS/V021/NOAH/G025/T3H/A20231211_1800,A20231211_1800,1702328400000,1702317600000,1.702695e+15
